# Dataton BC 2018

*Dirección de Capacidades Analíticas y Gobierno de Información, Grupo Bancolombia*<br>
Esta versión: *2018-10-17*<br>
Documentación tablas

## Naturaleza de los datos

Los datos entregados en este reto corresponden a transacciones realizadas por clientes persona del banco vía [PSE](https://www.pse.com.co/inicio). Estas transacciones, a diferencia de las transacciones realizadas vía POS, no cuentan con un código [MCC](https://en.wikipedia.org/wiki/Merchant_category_code) atado a la transacción, que permite conocer la categoría de comercio a la que pertence el establecimiento de comercio donde se realiza la transacción. Adicionalmente, muchas de estas transferencias por PSE corresponden a transferencias de pagos de servicios públicos, seguros, colegios, arrendamientos, y otros gastos que pueden ser denominados como gastos grandes. En el marco de un sistema de gestión de finanzas personales, poder categorizar adecuadamente estas transacciones que se realizan por PSE es de suma importancia para contar con una foto completa de la actividad de gastos de los clientes. Para este reto, los equipos participantes tendrán acceso a una muestra de transacciones PSE que corresponden a algo más de 300 mil clientes (persona), seleccionados de manera aleatoria. La tabla de transacciones cuenta con 11.8 millones de registros (uno para cada transacción), realizados entre septiembre de 2016 y octubre de 2018.

**NOTA** Los datos han pasado por un proceso relativamente simple de curación, pero se han dejado algunos ruidos en la calidad de éstos con el fin de que los equipos también lleven a cabo un proceso de inspección y limpieza.


## Categorización propuesta por el equipo de analítica de personas

En el Banco ya se han llevado a cabo esfuerzos por categorizar transacciones provenientes del canal POS (con tarjetas débito y crédito), lo cual ha incluído, entre otras cosas, una depuración y limpieza de los códigos MCC. A continuación mostramos, a manera de referencia, la categorización propuesta por el equipo.

1. Comida
2. Hogar
3. Cuidado personal
4. Entretenimiento
5. Educación
6. Transporte
7. Viajes
8. Ahorro
9. Pago de deudas
10. Ingresos
11. Retiros en efectivo
12. Mascotas
13. moda
14. Tecnología y comunicaciones
15. Otros

## NOTA IMPORTANTE

Recuerden que esta información aún contiene un elevado nivel de ruido. No solo no ha sido depurada de posibles datos atípicos (transacciones de valor muy elevado) fruto de errores o transacciones fallidas, sino que también cuenta con el ruido asociado al campo de referencia, donde se involucra el factor humano, ya que son campos de texto libre que pueden contener cualquier tipo de información.

Por seguridad, hemos eliminado cualquier número presente en dichos campos de referencia (cédulas, nits, montos, contratos, etc.).


In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
import matplotlib.animation as animation
import csv
matplotlib.style.use('seaborn')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
# from wordcloud import WordCloud

from mpl_toolkits.mplot3d import Axes3D

## Cargar el archivo de muestra  de textos procesado

In [49]:
transac_texts=[]
err_count=0
with open(
#     "transac_out_textos - copia.csv",
    "transac_out_textos.csv",
#     encoding="utf-8"
    'r'
) as dest_f:
    data_iter = csv.reader(dest_f,
                           delimiter = ",",
                           quoting=csv.QUOTE_NONE)
    for data in data_iter:
        try:
#             if len(data)==11:
#                 if random.random()<=prop_muestr:# se selecciona el caso que esté dentro del umbral del tamaño de la muestra
            transac_texts.append(data)
        except: #csv.Error as e:
            err_count+=1
            continue

In [50]:
transac_text_colnames=["id_trn_ach","ref","sector","subsector","descripcion"]

In [51]:
transac_descript=pd.DataFrame.from_records(transac_texts,columns=transac_text_colnames)
# transac_descript=pd.read_csv('transac_out_textos.csv')#,index_col=0)#"muest_1775488_proces.csv"
print(transac_descript.shape)
print(transac_descript.dtypes)
transac_descript.head(40)

(11592597, 5)
id_trn_ach     object
ref            object
sector         object
subsector      object
descripcion    object
dtype: object


,id_trn_ach,ref,sector,subsector,descripcion
0,230435642,CC,,,
1,222356110,Referencia: Contrato: Valor: CC,,,
2,309137749,CC,,,
3,324614737,CC,,,
4,235344690,MEDICINA PREPAGADA COLSANITAS CE,,,
5,320049316,RECAUDO COLSANITAS CC,,,
6,230519178,CC,,,
7,241307506,Pago de la Planilla Cesantias CEDULA DE CIUDAD...,SERVICIOS FINANCIEROS,OTROS SERVICIOS FINANCIEROS,Otras actividades auxiliares de las actividade...
8,316193109,CC,,,
9,282076350,Pago de la factura # CONJUNTO RESIDENCIAL PIET...,,,


## Cargar los textos y procesarlos en clusters según las referencias que tenemos

## Utilizar las transacciones con texto como elementos de aprendizaje para categorizar las transacciones

## Predecir las transacciones no anotadas

Utilizar las referencias como variables predictoras

tomar como criterios las dimensiones de temporalidad:

- dia de la semana
- hora del día
- mes del año

revisar ciclicidad
- trimestral
- semestral






In [57]:
print(transac_descript.shape)
pd.crosstab(transac_descript["sector"],columns="count")

(11592597, 5)


col_0,count
sector,
,8308234
AGROINDUSTRIA,521
COMERCIO,3769
CONSTRUCCION,25753
GOBIERNO,519376
MANUFACTURA INSUMOS,666
MEDIOS DE COMUNICACION,1173075
PERSONAS,5238
RECURSOS NATURALES,442485


In [58]:
11592597-8308234

3284363

In [52]:
pd.crosstab(index=[transac_descript["sector"],transac_descript["subsector"]],columns="count")

col_0                                                               count
sector                   subsector                                       
                                                                  8308234
AGROINDUSTRIA            ALIMENTOS CONCENTRADOS                       519
                         ARROZ                                          2
COMERCIO                 AUTOMOTORES                                  146
                         COMERCIO DE REPUESTOS                          1
                         COMERCIO DE VARIEDADES Y VESTUARIO          3413
                         DISTRIBUIDORES PRODUCTOS CONSUMO MASIVO      185
                         TEXTILES                                      24
CONSTRUCCION             EDIFICACIONES                               1732
                         OBRAS DE INFRAESTRUCTURA                   18572
                         TRANSPORTE AEREO                             113
                         TRANSPORTE TERRESTRE                        5336
GOBIERNO                 ADMINISTRACIÓN CENTRAL                    331829
                         ASEO                                         122
                         CAJAS DE COMPENSACIÓN                      39995
                         DEPARTAMENTOS                                533
                         EPS Y SALUD PREPAGADA (SALUD)              11759
                         ESTABLECIMIENTOS EDUCATIVOS                39607
                         IPS                                          344
                         MUNICIPIOS                                 95187
MANUFACTURA INSUMOS      AGROQUIMICOS                                   2
                         IMPRENTAS Y EDICION                          487
                         MADERA Y MUEBLES                               1
                         OTROS MANUFACTURA                            168
                         PLASTICO                                       1
                         PRODUCTOS CERAMICOS                            4
                         PRODUCTOS METALICOS                            3
MEDIOS DE COMUNICACION   MEDIOS                                      1309
                         TELEFONIA FIJA                            623746
                         VALOR AGREGADO                            548020
PERSONAS                 PERSONAS                                    5238
RECURSOS NATURALES       ELECTRICIDAD                              442484
                         OTRAS ACTIVIDADES MINERAS                      1
SERVICIOS FINANCIEROS    BANCOS                                   1041874
                         COMISIONISTAS DE BOLSA                        12
                         COMPANÍAS DE FINANCIAMIENTO COMERCIAL         21
                         FONDOS DE PENSIONES                          402
                         OTROS SERVICIOS FINANCIEROS                 5461
                         SEGUROS                                     8357
SERVICIOS NO FINANCIEROS SERVICIOS A EMPRESAS                       34535
                         SERVICIOS A PERSONAS                       22818